In [191]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import time
import random
import copy


In [192]:
# 现在要对5个州做一个总的用水用电的水分配方案（分别给多少水，给多少电 -> 存多少水）
# ai = 5个州的分别的总用水需求，bi = 5个州分别的用电需求 -> 转化为5个州的存水需求 wi
# 求最优的ai、wi，再计算得到最优的ai、bi
# 最优的评判：满足总和小于等于 M_v + P_v 的前提下，得分score()最高
# Ai = 实际总用水， Bi = 实际总用电
# (2015 data)
M_v = 4207.6251
P_v = 3212.9429
N = int(M_v + P_v)
Ai = eval("""[4632.39
20171.5
7849.86
2203.23
9412.49]""".replace('\n', ','))
Bi = eval("""316.5298945
480.7151231
147.2743259
94.57796014
151.905041""".replace('\n', ','))
Ai, Bi = np.array(Ai), np.array(Bi)
Ai = Ai / 1e3
Bi = Bi / 365
N

7420

In [193]:
# 因为目前只有两个水坝分别的水电转换的关系式，应该需要再拟合一个总的：y = 0.0010x + 26.2830
def w2e(x): return 0.0010 * x + 26.2830
def e2w(y): return (y - 26.2830) / 0.0010

m=1
def score(a, a_prime, m):
    return m / a_prime - m / a


In [194]:
print(Bi, e2w(Bi))
Ai = np.append(Ai, Bi)

print(Ai)


[0.86720519 1.31702773 0.4034913  0.2591177  0.41617819] [-25415.79480959 -24965.97226548 -25879.50869616 -26023.88230099
 -25866.82180548]
[ 4.63239    20.1715      7.84986     2.20323     9.41249     0.86720519
  1.31702773  0.4034913   0.2591177   0.41617819]


In [195]:
def init_random():
    x, tmp = [], []
    for i in range(9):
        ran = random.randint(0, N)
        if ran not in tmp and ran != 0 and ran != N:
            tmp.append(ran)
    x.append(tmp[0])
    for i in range(1, 9):
        x.append(tmp[i]-tmp[i-1])
    x.append()
    return x

def swap(x):
    new_x = [] * len(x)
    x2 = copy.copy(x)
    np.random.shuffle(x)
    bitlength = N.bit_length()
    for i in range(0, len(x)-1, 2):
        s = random.randint(0, bitlength)
        new_x[x2.index(x[i])] = bin(x[i])[2:].zfill(bitlength)[:s] + bin(x[i+1])[2:].zfill(bitlength)[s:]
        new_x[x2.index(x[i+1])] = bin(x[i+1])[2:].zfill(bitlength)[:s] + bin(x[i])[2:].zfill(bitlength)[s:]


In [196]:
x = init_random()
print(x)
swap(x)

[3708, 4977, 2726, 915, 4060, 6979, 6978, 2670, 4488]
1101101000011 1101101000010
1101101000010 1101101000011
0001110010011 1000110001000
0001110011000 1000110000011
1001101110001 0111111011100
1001101011100 0111111110001
0111001111100 0101010100110
0111010100110 0101001111100
